## 1. Check GPU Availability

In [7]:
import torch

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    print("⚠️ No GPU detected! Go to Runtime → Change runtime type → GPU")

PyTorch version: 2.9.0+cu126
CUDA available: True
GPU: Tesla T4
GPU Memory: 15.8 GB


## 2. Mount Google Drive (for persistent storage)

In [9]:
import sys
import os

# Detect if running in Google Colab
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')
    
    # Create project directory in Drive
    os.makedirs('/content/drive/MyDrive/ML_Sentiment_Analysis/checkpoints', exist_ok=True)
    os.makedirs('/content/drive/MyDrive/ML_Sentiment_Analysis/results', exist_ok=True)
    os.makedirs('/content/drive/MyDrive/ML_Sentiment_Analysis/plots', exist_ok=True)
    
    BASE_DIR = "/content/drive/MyDrive/ML_Sentiment_Analysis"
    print("✅ Google Drive mounted!")
else:
    # Running locally - use project directory
    BASE_DIR = os.path.dirname(os.path.dirname(os.path.abspath(".")))
    if not os.path.exists(os.path.join(BASE_DIR, "src")):
        # Fallback: use current working directory's parent
        BASE_DIR = os.path.dirname(os.getcwd())
    
    os.makedirs(os.path.join(BASE_DIR, 'checkpoints'), exist_ok=True)
    os.makedirs(os.path.join(BASE_DIR, 'results'), exist_ok=True)
    os.makedirs(os.path.join(BASE_DIR, 'plots'), exist_ok=True)
    
    print(f"📁 Running locally. Base directory: {BASE_DIR}")

print(f"IN_COLAB: {IN_COLAB}")

ValueError: mount failed

## 3. Clone/Upload Project

In [ ]:
import zipfile
import os

if IN_COLAB:
    # Option A: Clone from GitHub (uncomment and replace with your repo)
    # !git clone https://github.com/yourusername/sentiment_analysis.git
    # %cd sentiment_analysis

    # Option B: Upload from local (run this cell, then upload the zip)
    from google.colab import files
    
    print("Upload your project as a .zip file:")
    uploaded = files.upload()
    
    # Extract the uploaded zip
    for filename in uploaded.keys():
        if filename.endswith('.zip'):
            with zipfile.ZipFile(filename, 'r') as zip_ref:
                zip_ref.extractall('/content/')
            print(f"✅ Extracted {filename}")
            # Find the extracted directory
            extracted_dirs = [d for d in os.listdir('/content/') if os.path.isdir(f'/content/{d}') and d != 'drive' and d != 'sample_data']
            if extracted_dirs:
                os.chdir(f'/content/{extracted_dirs[0]}')
                print(f"📁 Working directory: {os.getcwd()}")
else:
    # Running locally - change to project root
    project_root = os.path.dirname(os.path.dirname(os.path.abspath(".")))
    if os.path.exists(os.path.join(project_root, "src")):
        os.chdir(project_root)
    else:
        # Try parent of current directory
        project_root = os.path.dirname(os.getcwd())
        if os.path.exists(os.path.join(project_root, "src")):
            os.chdir(project_root)
    
    print(f"📁 Working directory: {os.getcwd()}")

In [ ]:
# Verify project structure
!ls -la
print("\n📂 Source directory:")
!ls -la src/

total 16
drwxr-xr-x 1 root root 4096 Dec  9 14:41 .
drwxr-xr-x 1 root root 4096 Jan  3 16:50 ..
drwxr-xr-x 4 root root 4096 Dec  9 14:41 .config
drwxr-xr-x 1 root root 4096 Dec  9 14:42 sample_data

📂 Source directory:
ls: cannot access 'src/': No such file or directory


## 4. Install Dependencies

In [ ]:
# Install from requirements.txt if exists
!pip install -q -r requirements.txt 2>/dev/null || echo "No requirements.txt found"

# Install core dependencies
!pip install -q torch pandas scikit-learn matplotlib seaborn spacy

# Download Romanian spaCy model
!python -m spacy download ro_core_news_sm -q

print("✅ Dependencies installed!")

## 5. Download Dataset

In [ ]:
import os
import pandas as pd

# Create data directories
os.makedirs('data/raw', exist_ok=True)
os.makedirs('data/processed', exist_ok=True)

# Download ro_sent dataset
TRAIN_URL = "https://raw.githubusercontent.com/dumitrescustefan/Romanian-Transformers/examples/examples/sentiment_analysis/ro/train.csv"
TEST_URL = "https://raw.githubusercontent.com/dumitrescustefan/Romanian-Transformers/examples/examples/sentiment_analysis/ro/test.csv"

!wget -q -O data/raw/train.csv "{TRAIN_URL}" 2>/dev/null || echo "Downloading train.csv..."
!wget -q -O data/raw/test.csv "{TEST_URL}" 2>/dev/null || echo "Downloading test.csv..."

# Check if download succeeded, if not try alternative method
if not os.path.exists('data/raw/train.csv') or os.path.getsize('data/raw/train.csv') < 1000:
    print("Trying alternative download method...")
    # Use datasets library as fallback
    !pip install -q datasets
    from datasets import load_dataset
    dataset = load_dataset("dumitrescustefan/ro_sent")
    dataset['train'].to_pandas().to_csv('data/raw/train.csv', index=False)
    dataset['test'].to_pandas().to_csv('data/raw/test.csv', index=False)

# Verify download
train_df = pd.read_csv('data/raw/train.csv')
test_df = pd.read_csv('data/raw/test.csv')
print(f"✅ Train samples: {len(train_df)}")
print(f"✅ Test samples: {len(test_df)}")
print(f"\nColumns: {train_df.columns.tolist()}")
print(f"\nLabel distribution (train):")
print(train_df['label'].value_counts())

In [ ]:
# Create train/val/test split
from sklearn.model_selection import train_test_split

# Split train into train/val (90/10)
train_data, val_data = train_test_split(
    train_df, 
    test_size=0.1, 
    random_state=42, 
    stratify=train_df['label']
)

# Save processed splits
train_data.to_csv('data/processed/train.csv', index=False)
val_data.to_csv('data/processed/val.csv', index=False)
test_df.to_csv('data/processed/test.csv', index=False)

print(f"✅ Train: {len(train_data)} | Val: {len(val_data)} | Test: {len(test_df)}")

## 6. Run Experiments

Configure and run your training experiments below.

In [ ]:
# Paths for persistent storage - dynamically set based on environment
if IN_COLAB:
    CHECKPOINT_DIR = "/content/drive/MyDrive/ML_Sentiment_Analysis/checkpoints"
    RESULTS_DIR = "/content/drive/MyDrive/ML_Sentiment_Analysis/results"
    PLOTS_DIR = "/content/drive/MyDrive/ML_Sentiment_Analysis/plots"
else:
    CHECKPOINT_DIR = os.path.join(os.getcwd(), "checkpoints")
    RESULTS_DIR = os.path.join(os.getcwd(), "results")
    PLOTS_DIR = os.path.join(os.getcwd(), "plots")
    
    # Create directories if they don't exist
    os.makedirs(CHECKPOINT_DIR, exist_ok=True)
    os.makedirs(RESULTS_DIR, exist_ok=True)
    os.makedirs(PLOTS_DIR, exist_ok=True)

print(f"Checkpoint dir: {CHECKPOINT_DIR}")
print(f"Results dir: {RESULTS_DIR}")
print(f"Plots dir: {PLOTS_DIR}")

### Experiment 1: LSTM Baseline

In [ ]:
import subprocess

cmd = [
    "python", "-m", "src.run_experiment",
    "--model", "lstm",
    "--hidden_dim", "256",
    "--num_layers", "2",
    "--epochs", "20",
    "--batch_size", "64",
    "--lr", "0.001",
    "--device", "cuda" if IN_COLAB else "cpu",
    "--experiment_name", "lstm_baseline",
    "--checkpoint_dir", CHECKPOINT_DIR,
    "--results_dir", RESULTS_DIR,
    "--plots_dir", PLOTS_DIR
]

print(f"Running: {' '.join(cmd)}")
subprocess.run(cmd)

### Experiment 2: BiLSTM with Attention

In [ ]:
import subprocess

cmd = [
    "python", "-m", "src.run_experiment",
    "--model", "bilstm_attention",
    "--hidden_dim", "256",
    "--num_layers", "2",
    "--epochs", "20",
    "--batch_size", "64",
    "--lr", "0.001",
    "--device", "cuda" if IN_COLAB else "cpu",
    "--experiment_name", "bilstm_attention",
    "--checkpoint_dir", CHECKPOINT_DIR,
    "--results_dir", RESULTS_DIR,
    "--plots_dir", PLOTS_DIR
]

print(f"Running: {' '.join(cmd)}")
subprocess.run(cmd)

### Experiment 3: LSTM with Augmentation

In [ ]:
import subprocess

cmd = [
    "python", "-m", "src.run_experiment",
    "--model", "lstm",
    "--hidden_dim", "256",
    "--num_layers", "2",
    "--epochs", "20",
    "--batch_size", "64",
    "--lr", "0.001",
    "--augment", "random_swap",
    "--aug_prob", "0.1",
    "--device", "cuda" if IN_COLAB else "cpu",
    "--experiment_name", "lstm_augmented",
    "--checkpoint_dir", CHECKPOINT_DIR,
    "--results_dir", RESULTS_DIR,
    "--plots_dir", PLOTS_DIR
]

print(f"Running: {' '.join(cmd)}")
subprocess.run(cmd)

### Experiment 4: Simple RNN (for comparison)

In [ ]:
import subprocess

cmd = [
    "python", "-m", "src.run_experiment",
    "--model", "simple_rnn",
    "--hidden_dim", "256",
    "--num_layers", "2",
    "--epochs", "20",
    "--batch_size", "64",
    "--lr", "0.001",
    "--device", "cuda" if IN_COLAB else "cpu",
    "--experiment_name", "simple_rnn_baseline",
    "--checkpoint_dir", CHECKPOINT_DIR,
    "--results_dir", RESULTS_DIR,
    "--plots_dir", PLOTS_DIR
]

print(f"Running: {' '.join(cmd)}")
subprocess.run(cmd)

## 7. View Results

In [ ]:
import json
import pandas as pd
from pathlib import Path

# Load all experiment summaries
results_dir = Path(RESULTS_DIR)
summaries = []

if results_dir.exists():
    for exp_dir in results_dir.iterdir():
        summary_file = exp_dir / 'summary.json'
        if summary_file.exists():
            with open(summary_file) as f:
                summary = json.load(f)
                summary['experiment'] = exp_dir.name
                summaries.append(summary)

if summaries:
    df = pd.DataFrame(summaries)
    print("\n" + "="*60)
    print("EXPERIMENT RESULTS SUMMARY")
    print("="*60)
    display(df[['experiment', 'model', 'best_val_f1', 'best_val_acc', 'epochs_trained']].sort_values('best_val_f1', ascending=False))
else:
    print("No results found yet. Run experiments first!")

In [ ]:
# Display plots from experiments
from IPython.display import Image, display
from pathlib import Path

plots_dir = Path(PLOTS_DIR)
if plots_dir.exists():
    for exp_dir in plots_dir.iterdir():
        print(f"\n📊 {exp_dir.name}")
        print("-" * 40)
        for plot in exp_dir.glob('*.png'):
            print(f"\n{plot.name}:")
            display(Image(filename=str(plot), width=600))
else:
    print(f"Plots directory not found: {PLOTS_DIR}")

## 8. Download Results to Local Machine

In [ ]:
if IN_COLAB:
    import subprocess
    # Zip and download all results
    subprocess.run(["zip", "-r", "/content/experiment_results.zip", RESULTS_DIR, PLOTS_DIR, CHECKPOINT_DIR])
    
    from google.colab import files
    files.download('/content/experiment_results.zip')
    print("✅ Results downloaded!")
else:
    print(f"Results are saved locally at:")
    print(f"  - Results: {RESULTS_DIR}")
    print(f"  - Plots: {PLOTS_DIR}")
    print(f"  - Checkpoints: {CHECKPOINT_DIR}")

## 🔧 Custom Experiment Runner

Use this cell to run custom experiments with your own parameters:

In [ ]:
import subprocess

# Custom experiment configuration
CONFIG = {
    'model': 'lstm',              # simple_rnn, stacked_rnn, lstm, bilstm_attention, lstm_batchnorm, stacked_lstm
    'hidden_dim': 256,
    'num_layers': 2,
    'dropout': 0.3,
    'bidirectional': False,       # Only for lstm, lstm_batchnorm, stacked_lstm
    'pooling': 'last',            # last, max, mean
    
    'epochs': 20,
    'batch_size': 64,
    'lr': 0.001,
    'optimizer': 'adamw',         # adam, adamw, sgd
    'scheduler': 'plateau',       # plateau, cosine, none
    'weight_decay': 1e-5,
    'gradient_clip': 1.0,
    
    'augment': None,              # random_swap, random_delete, random_insert, random_shuffle, eda, None
    'aug_prob': 0.1,
    
    'early_stopping': 5,
    'experiment_name': 'custom_experiment',
}

# Build command
cmd = ["python", "-m", "src.run_experiment"]
for key, value in CONFIG.items():
    if value is not None and value is not False:
        if isinstance(value, bool):
            cmd.append(f"--{key}")
        else:
            cmd.extend([f"--{key}", str(value)])

cmd.extend(["--device", "cuda" if IN_COLAB else "cpu"])
cmd.extend(["--checkpoint_dir", CHECKPOINT_DIR])
cmd.extend(["--results_dir", RESULTS_DIR])
cmd.extend(["--plots_dir", PLOTS_DIR])

print("Running command:")
print(" ".join(cmd))
print("\n" + "="*60 + "\n")

subprocess.run(cmd)